# Pillar 2 · Compliance and Regulatory Frameworks

Use this notebook to map Azure AI agent workflows to legal, regulatory, and internal compliance controls.

## Learning Objectives

- Identify regulatory obligations for agent data flows
- Instrument logging and retention policies within the agent stack
- Define approval checkpoints for high-risk scenarios

## Compliance Blueprint

1. **Data Inventory**: Follow ingestion patterns from `17_agent_ops/01_agent/03_agent_db_demo.ipynb` to document personal data processed by each agent.
2. **Retention Controls**: Configure Cosmos DB TTL policies and align with privacy requirements (e.g., GDPR, CCPA).
3. **Access Governance**: Reuse authentication patterns from `agent_utils.AgentManager` to enforce least-privilege access for maintenance tooling.
4. **Regulatory Evidence**: Generate reports summarizing consent status, data residency, and DPIA outcomes.

## Controls Checklist

- Data protection impact assessment completed
- Retention policies validated in production
- Access reviews logged and approved

## Configure PATH for Azure CLI

Ensure the Azure CLI is accessible in the notebook kernel's PATH.

In [ ]:
import os
import shutil

new_path_entry = "/opt/homebrew/bin"  # Replace with the directory you want to add
current_path = os.environ.get('PATH', '')

if new_path_entry not in current_path.split(os.pathsep):
    os.environ['PATH'] = new_path_entry + os.pathsep + current_path
    print(f"Updated PATH for this session: {os.environ['PATH']}")
else:
    print(f"PATH already contains {new_path_entry}: {current_path}")

# You can then verify with shutil.which again
print(f"Location of 'az' found by kernel now: {shutil.which('az')}")

In [ ]:
import os
import sys
from dotenv import load_dotenv

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'utils'))
from agent_db import AgentDB

# Load environment variables
load_dotenv("../.env")

print("✅ Imports successful")

# AgentDB uses AZURE_COSMOS_ENDPOINT from environment
if not os.getenv("AZURE_COSMOS_ENDPOINT"):
    print("⚠️  AZURE_COSMOS_ENDPOINT not set. Set it in .env.local file:")
    print("   AZURE_COSMOS_ENDPOINT=https://your-cosmos-account.documents.azure.com:443/")
    raise ValueError("AZURE_COSMOS_ENDPOINT environment variable is required")

db = AgentDB()  # Uses default database "Acsp" and container "Agents"
print(f"✅ Connected to container: {db.container.id}")

In [ ]:
# Flag agent records that reference personal data fields
from typing import List
import json


def list_agents_with_personal_data(fields: List[str]):
    records = db.list_agents()
    if not records:
        print("No agent metadata found.")
        return

    print(
        f"🔍 Scanning {len(records)} agents for personal data fields: {fields}\n")

    for item in records:
        # Fields to check for personal data
        searchable_fields = {
            'name': item.get('name', ''),
            'description': item.get('description', ''),
            'instruction': item.get('instruction', ''),
            'content': item.get('content', ''),
            'sample_prompts': json.dumps(item.get('samplePrompts', [])),
            'knowledge_base': item.get('knowledgeBase', '')
        }

        # Check each field for personal data keywords
        hits = {}
        for field_name, field_value in searchable_fields.items():
            if field_value:
                field_lower = str(field_value).lower()
                found = [keyword for keyword in fields if keyword.lower()
                         in field_lower]
                if found:
                    hits[field_name] = found

        if hits:
            print(f"⚠️  Agent: {item['id']} ({item.get('name', 'unknown')})")
            print(f"   Category: {item.get('category', 'N/A')}")
            print(f"   Status: {item.get('status', 'N/A')}")
            for field, keywords in hits.items():
                print(f"   - Found in '{field}': {keywords}")
            print()


list_agents_with_personal_data(
    ["email", "address", "phone", "birthdate", "ssn", "credit card"])

## Next Notebook

Continue with `04_deployment_monitoring.ipynb` to operationalize agent deployment safeguards.